# Extract Frames from surgical videos at 1 fps

This script processes a list of patient directories (e.g. '10XX'), each containing multiple video segments (.mp4) representing parts of the same surgical procedure.

For each video, it extracts frames at 1 fps using ffmpeg and stores the output
in a structured subdirectory inside each patient's folder:
Patients/{patient_id}/frames/{video_segment}/frame.png

(e.g. /Patients/1001/frames/1001_01/1001_01_001.png)

Each frame is resized to 456x256 and saved as .png.

In [ ]:
import os
import subprocess
import shutil


In [ ]:
# Base directory containing patient folders
base_dir = "/home/Patients/"

# List of patient IDs to process
patient_list = [name for name in os.listdir(base_dir)
                if os.path.isdir(os.path.join(base_dir, name)) and name.isdigit()]

# Process each patient folder
for patient_id in patient_list:
    patient_video_dir = os.path.join(base_dir, patient_id)
    output_base_dir = os.path.join(patient_video_dir, "frames")
    os.makedirs(output_base_dir, exist_ok=True)

    # Iterate through all video files in the patient's folder
    for video_file in os.listdir(patient_video_dir):
        if video_file.endswith(".mp4"):
            video_path = os.path.join(patient_video_dir, video_file)
            video_name = os.path.splitext(video_file)[0]
            frame_output_dir = os.path.join(output_base_dir, video_name)
            os.makedirs(frame_output_dir, exist_ok=True)

            # Use ffmpeg to extract 1 frame per second, resized to 456x256
            command = [
                "ffmpeg",
                "-i", video_path,
                "-vf", "fps=1,scale=456:256",
                os.path.join(frame_output_dir, f"{video_name}_%04d.png")
            ]
            subprocess.run(command)

    print(f"Frame extraction completed for patient {patient_id}")

In [ ]:
# Source directory containing patient folders
source_dir = '/home/Patients'

# Destination directory for all frames
destination_dir = '/home/images'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Iterate through each patient folder in the source directory
for patient_folder in os.listdir(source_dir):
    patient_path = os.path.join(source_dir, patient_folder)
    
    # Check that the path is a directory
    if os.path.isdir(patient_path):
        
        # Locate the 'frames' subfolder within the patient folder
        frames_folder = os.path.join(patient_path, 'frames')
        
        if os.path.isdir(frames_folder):
            # Iterate through each video-specific frame folder
            for frame_folder in os.listdir(frames_folder):
                frame_folder_path = os.path.join(frames_folder, frame_folder)
                
                # Ensure this is a directory containing image files
                if os.path.isdir(frame_folder_path):
                    for image_file in os.listdir(frame_folder_path):
                        # Full path to the image
                        image_path = os.path.join(frame_folder_path, image_file)
                        
                        # Only copy valid image files
                        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                            # Define the new path in the destination directory
                            new_image_path = os.path.join(destination_dir, image_file)
                            
                            # Copy the image file to 'image' folder
                            shutil.copy(image_path, new_image_path)

                            print(f"Copied {image_file} to {destination_dir}")